In [1]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

/home/egor/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/egor/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/egor/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/egor/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/egor/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from num

# Подход 1: градиентный бустинг "в лоб

#### Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).


In [2]:
data = pd.read_csv('features.csv')
data.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,4,3,1,13.0,2130,0,0,1830,0,63
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
data.drop(['match_id',
           'duration',
           'tower_status_radiant',
           'tower_status_dire',
            'barracks_status_radiant',
            'barracks_status_dire'], 1, inplace=True)

#### Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [5]:
print(data.head())
data.count()[  data.count() < len(data)]

   start_time  lobby_type  r1_hero  r1_level  r1_xp  r1_gold  r1_lh  r1_kills  \
0  1430198770           7       11         5   2098     1489     20         0   
1  1430220345           0       42         4   1188     1033      9         0   
2  1430227081           7       33         4   1319     1270     22         0   
3  1430263531           1       29         4   1779     1056     14         0   
4  1430282290           7       13         4   1431     1090      8         1   

   r1_deaths  r1_items     ...       radiant_first_ward_time  \
0          0         7     ...                          35.0   
1          1        12     ...                         -20.0   
2          0        12     ...                         -39.0   
3          0         5     ...                         -30.0   
4          0         8     ...                          46.0   

   dire_bottle_time  dire_courier_time  dire_flying_courier_time  \
0             103.0              -84.0                     2

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Ответ: пропуски имеют признаки: first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward_time, dire_bottle_time, dire_courier_time, dire_flying_courier_time, dire_first_ward_time

Ответ: признаки first_blood_time, first_blood_team, first_blood_player1, first_blood_player2 имеют пропущенные значения в случае, когда событие "первая кровь" не успело произойти за первые 5 минут

#### Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [22]:
data.fillna(0, inplace=True)

Какой столбец содержит целевую переменную? Запишите его название.

In [38]:
y = data['radiant_win'].values
X = data[[x for x in data.columns if x != 'radiant_win']].values

Ответ: столбец содержащий целевую переменную называется: radiant_win

#### Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?


Ответ: средний AUC под ROC-кривой составляет 0,63 при отклонении в 0

In [24]:
%%time 
params = {'n_estimators': [10, 20, 30]}
model = GridSearchCV(GradientBoostingClassifier(), params, 
                     scoring=make_scorer(roc_auc_score, needs_proba=True),
                     cv=KFold(n_splits=5, shuffle=True))
model.fit(X, y)

CPU times: user 4min 27s, sys: 1.88 s, total: 4min 29s
Wall time: 4min 30s


In [25]:
model.cv_results_['mean_test_score']

array([0.66503854, 0.68197451, 0.68949619])

Ответ: да, имеет смысл увеличивать количество деревьев, поскольку с увеличением количества деревьев метрика качества растёт

In [26]:
print('Learning time on 5 folds', "{0: .0f}".format(model.cv_results_['mean_fit_time'][2] * 5))

Learning time on 5 folds  115


Ответ: кросс валидация заняла 115 секунды

Ответ: для ускорения обучения градиентного бустинга, можно уменьшить размерность данных (PCA, отбор признаков), воспользоваться LightGBM,уменьшить глубину деревьев

# Подход 2: логистическая регрессия

#### Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [39]:
%%time
params = {'model__C': [.0000001, .001, .01, .1, 1, 10]}
model = Pipeline([('prep', StandardScaler()), ('model', LogisticRegression(solver='lbfgs'))])
gs = GridSearchCV(model, params, 
                     scoring=make_scorer(roc_auc_score, needs_proba=True),
                     cv=KFold(n_splits=5, shuffle=True))
gs.fit(X, y)

CPU times: user 1min 29s, sys: 46.6 s, total: 2min 16s
Wall time: 46 s


In [40]:
gs.cv_results_['mean_test_score']

array([0.6871366 , 0.71617393, 0.71639339, 0.71637933, 0.71637711,
       0.71637703])

Ответ: Логистическая регрессия дала лучше результаты, чем градиентный бустинг, поскольку мы имели данныес большим количеством признаков, дл достежения такого же качества градиентным бустингом стоило строить больше количество деревьев

In [41]:
gs.cv_results_['mean_fit_time'] * 5

array([ 3.00382614,  4.87295771,  6.57772636,  7.13391614,  7.7323463 ,
       10.26800919])

Ответ: да, логистическая регрессия работает бстрее градиентного бустинга (всего 8 сек на 5 фолдах)

#### Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [42]:
%%time
X = data[[x for x in data.columns if x not in ['lobby_type', 'radiant_win'] 
                                     +['r'+str(i)+'_hero' for i in range(1, 6)]
                                     +['d'+str(i)+'_hero' for i in range(1, 6)]]].values
params = {'model__C': [.0000001, .001, .01, .1, 1, 10]}
model = Pipeline([('prep', StandardScaler()), ('model', LogisticRegression(solver='lbfgs'))])
gs = GridSearchCV(model, params, 
                     scoring=make_scorer(roc_auc_score, needs_proba=True),
                     cv=KFold(n_splits=5, shuffle=True))
gs.fit(X, y)

CPU times: user 1min 27s, sys: 42.9 s, total: 2min 10s
Wall time: 40 s


In [43]:
gs.cv_results_['mean_test_score']

array([0.68715987, 0.7160577 , 0.71621794, 0.7161915 , 0.71618721,
       0.7161866 ])

Ответ: Нет, качество особо не улучшилось, поскольку были удалены значимые категориальные признаки, которые со

#### На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [9]:
heroes_lists = [list(data['r'+str(i)+'_hero'].unique()) for i in range(1,6)]
heroes_lists += [list(data['d'+str(i)+'_hero'].unique()) for i in range(1,6)]
heroes_list = []
for heroes in heroes_lists:
    heroes_list += heroes
len(set(heroes_list))

108

Ответ:в играх принимало участие 108 различных идентификаторов героев

#### Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [10]:
dummy_data = pd.DataFrame(data=np.zeros((len(X), 108)),columns=['hero_%d' % i for i in set(heroes_list)])
for i in range(1, 6):
    dummy_data += (pd.get_dummies(data['r'+str(i)+'_hero'], prefix='hero',dtype=int) 
    - pd.get_dummies(data['d'+str(i)+'_hero'], prefix='hero', dtype=int))
data_new = pd.concat([data, dummy_data], axis=1)

In [27]:
f = [x for x in data_new.columns if x not in ['lobby_type', 'radiant_win'] 
                                     +['r'+str(i)+'_hero' for i in range(1, 6)]
                                     +['d'+str(i)+'_hero' for i in range(1, 6)]]
X = data_new[f].values

#### Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [12]:
%%time
params = {'model__C': [.0000001, .001, .01, .1, 1, 10]}

model = Pipeline([('prep', StandardScaler()), ('model', LogisticRegression(solver='lbfgs'))])
gs = GridSearchCV(model, params, 
                     scoring=make_scorer(roc_auc_score, needs_proba=True),
                     cv=KFold(n_splits=5, shuffle=True))
gs.fit(X, y)

CPU times: user 2min 17s, sys: 55.5 s, total: 3min 13s
Wall time: 1min 5s


In [13]:
gs.cv_results_['mean_test_score']

array([0.70075982, 0.75163592, 0.75196211, 0.75192237, 0.7519168 ,
       0.7519165 ])

Ответ: качество повысилось, поскольку мы добавили информацию из категориальных признаков

#### Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [15]:
test = pd.read_csv('data/features_test.csv')
test.fillna(0, inplace=True)
dummy_data = pd.DataFrame(data=np.zeros((len(test), 108)),columns=['hero_%d' % i for i in set(heroes_list)])
for i in range(1, 6):
    dummy_data += (pd.get_dummies(test['r'+str(i)+'_hero'], prefix='hero',dtype=int) 
    - pd.get_dummies(test['d'+str(i)+'_hero'], prefix='hero', dtype=int))
test_new = pd.concat([test, dummy_data], axis=1)

In [35]:
test_prediction = gs.best_estimator_.predict_proba(test_new[f].values)[:, 1]
test_prediction

array([0.8300046 , 0.75819018, 0.19446202, ..., 0.24906836, 0.63917384,
       0.44034014])

In [36]:
test_prediction.min(), test_prediction.max()

(0.00840521274673193, 0.9963531262354892)

Ответ: минимальной значение прогноза 0.008, максимальное 0.996